original caco2 data set consists of 906 molecule instances with corresponding SMILES representations and their permeability values.

use `rdkit` amd `guache1` modules to acess five extra molecue representations as objectives and other features such as `ecfp_fingerprints` and `fragments` as domain.

In [10]:
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem import MolFromSmiles
from rdkit.Chem import  QED
from rdkit.Chem import  rdMolDescriptors
from rdkit.Chem import Descriptors
import numpy as np
import pandas as pd
# from gauche.dataloader import DataLoaderMP
from gauche.dataloader import MolPropLoader
from gauche.representations.fingerprints import (
    # ecfp_fingerprints,
    fragments,
    mqn_features,
    )
from gauche.representations.strings import bag_of_characters
from gauche.representations.graphs import molecular_graphs

Domain: fragprints

target:

- meaured log (solubility)
- logP
- drug-likeliness
- TPSA

In [26]:
original = pd.read_csv('ESOL_dataset.txt', delimiter= ',')
original = original.dropna()
smiles = original['SMILES']

#molecule objects in rdkit
molecules = [MolFromSmiles(s) for s in smiles]

#define Morgen generator
generator = GetMorganGenerator(radius = 3, fpSize = 2048)

#get finger prints
fps = [generator.GetFingerprint(mol) for mol in molecules]
fps_array = np.array(fps)

fragments

shape: 1144 by 85

In [28]:
#return np.array 
frag = fragments(smiles= smiles)

In [29]:
frag.shape

(1144, 85)

Synthesized Domain

1144, 2133

In [30]:
domain = np.concatenate((fps_array, frag), axis = 1)

In [38]:
import torch
domain = torch.tensor(domain)

Targets

QED score

In [17]:
qed = np.array([QED.qed(mol) for mol in molecules]).reshape((1144,-1))

TPSA

In [21]:
tpsa = np.array([rdMolDescriptors.CalcTPSA(mol) for mol in molecules]).reshape((1144,-1))

Log solubility

In [32]:
logSOL = np.array(original['measured log(solubility:mol/L)']).reshape((1144,-1))

In [36]:
logp  = np.array([Descriptors.MolLogP(mol) for mol in molecules]).reshape((1144,-1))

In [37]:
target = np.concatenate((logp, qed, tpsa, logSOL), axis = 1)

In [43]:
target = torch.tensor(target)
